In [10]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from keras import regularizers

In [11]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [12]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [13]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001, optomizer='sgd', activation='relu'):
    print('kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
        kernel_sizes, 
        neurons_bk1, 
        neurons_bk2, 
        learn_rate,
        optomizer,
        activation))
    
    model = Sequential()

    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1), kernel_regularizer=regularizers.l1(0.001)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same", kernel_regularizer=regularizers.l1(0.001)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(500))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())

    model.add(Dense(28))
    model.add(Activation("softmax"))
    
#     model.summary()
    if (optomizer == 'sgd'):
        model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=learn_rate), metrics=["accuracy"])
    else:
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learn_rate), metrics=["accuracy"])
    return model

In [14]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [15]:

kernel_sizes=[3, 5]
neurons_bk1=[10, 20, 30]
neurons_bk2=[50, 100]
learn_rate=[0.01, 0.005]
optomizer=['sgd', 'adam']
activation=['relu', 'leakrelu']



In [16]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 20
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [17]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
                for opt in optomizer:
                    for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer=opt,
                                            activation=act
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    opt,
                                    act)
                            print ('Best F1-Score')
                            
                        del model
                        del history
                        K.backend.clear_session()
                        K.backend.get_session()
                
                        print ('------ END MODEL ------\n\n')

print(printer)

kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=3, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 14s 36ms/step - loss: 15.8058 - acc: 0.0383 - val_loss: 15.8610 - val_acc: 0.0357
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.9751 - acc: 0.0281 - val_loss: 15.8439 - val_acc: 0.0357
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.5883 - acc: 0.0510 - val_loss: 15.8274 - val_acc: 0.0357
Epoch 4/20
392/392 [==============================] - 6s 15ms/step - loss: 15.8598 - acc: 0.0332 - val_loss: 15.8114 - val_acc: 0.0357
F1-Score = 0.0352773826458037
Best F1-Score
------ END MODEL ------


kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.7540 - acc: 0.0383 - val_loss: 15.8042 - val_acc: 0.0357
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.6305 - acc: 0.0459 - val_loss: 15.7872 - val_acc: 0.0357
Epoch 3/20


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=3, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.7644 - acc: 0.0434 - val_loss: 15.8521 - val_acc: 0.0368
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.6892 - acc: 0.0459 - val_loss: 15.8191 - val_acc: 0.0368
Epoch 3/20
392/392 [==============================] - 8s 21ms/step - loss: 15.8216 - acc: 0.0357 - val_loss: 15.7880 - val_acc: 0.0368
Epoch 4/20
392/392 [==============================] - 8s 21ms/step - loss: 15.5447 - acc: 0.0510 - val_loss: 15.7587 - val_acc: 0.0368
F1-Score = 0.03613086770981508
------ END MODEL ------


kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.8369 - acc: 0.0408 - val_loss: 15.9649 - val_acc: 0.0322
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.8499 - acc: 0.0383 - val_loss: 15.9318 - val_acc: 0.0322
Epoch 3/20
392/392 [====

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=5, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 16.8104 - acc: 0.0306 - val_loss: 16.6955 - val_acc: 0.0351
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 16.6398 - acc: 0.0357 - val_loss: 16.6050 - val_acc: 0.0351
Epoch 3/20
392/392 [==============================] - 8s 21ms/step - loss: 16.3465 - acc: 0.0485 - val_loss: 16.5201 - val_acc: 0.0351
Epoch 4/20
392/392 [==============================] - 8s 21ms/step - loss: 16.3054 - acc: 0.0459 - val_loss: 16.4405 - val_acc: 0.0351
F1-Score = 0.034139402560455195
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 23ms/step - loss: 16.5512 - acc: 0.0255 - val_loss: 16.2773 - val_acc: 0.0415
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 16.4905 - acc: 0.0255 - val_loss: 16.1886 - val_acc: 0.0415
Epoch 3/20
392/392 [===

392/392 [==============================] - 8s 20ms/step - loss: 15.6721 - acc: 0.0281 - val_loss: 15.4819 - val_acc: 0.0398
F1-Score = 0.031294452347083924
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 7s 17ms/step - loss: 16.5202 - acc: 0.0281 - val_loss: 16.3787 - val_acc: 0.0368
Epoch 2/20
392/392 [==============================] - 8s 20ms/step - loss: 16.5375 - acc: 0.0255 - val_loss: 16.3311 - val_acc: 0.0368
Epoch 3/20
392/392 [==============================] - 7s 18ms/step - loss: 16.3262 - acc: 0.0357 - val_loss: 16.2850 - val_acc: 0.0368
Epoch 4/20
392/392 [==============================] - 7s 17ms/step - loss: 16.2397 - acc: 0.0383 - val_loss: 16.2405 - val_acc: 0.0368
F1-Score = 0.03869132290184922
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activation=leakrelu
Epoch 1

Epoch 2/20
392/392 [==============================] - 12s 31ms/step - loss: 15.4878 - acc: 0.0536 - val_loss: 15.6630 - val_acc: 0.0351
Epoch 3/20
392/392 [==============================] - 12s 30ms/step - loss: 15.6458 - acc: 0.0332 - val_loss: 15.5842 - val_acc: 0.0351
Epoch 4/20
392/392 [==============================] - 12s 30ms/step - loss: 15.5208 - acc: 0.0383 - val_loss: 15.5643 - val_acc: 0.0351
F1-Score = 0.032432432432432434
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 7s 17ms/step - loss: 16.3662 - acc: 0.0459 - val_loss: 16.4023 - val_acc: 0.0392
Epoch 2/20
392/392 [==============================] - 6s 17ms/step - loss: 16.7781 - acc: 0.0306 - val_loss: 16.9617 - val_acc: 0.0345
Epoch 3/20
392/392 [==============================] - 6s 16ms/step - loss: 16.8725 - acc: 0.0357 - val_loss: 16.8238 - val_acc: 0.0345
Epoch 4/20
392/392 [=====

Epoch 1/20
392/392 [==============================] - 13s 34ms/step - loss: 17.4691 - acc: 0.0357 - val_loss: 16.3455 - val_acc: 0.0316
Epoch 2/20
392/392 [==============================] - 12s 31ms/step - loss: 15.9148 - acc: 0.0332 - val_loss: 15.7237 - val_acc: 0.0316
Epoch 3/20
392/392 [==============================] - 34s 86ms/step - loss: 15.7231 - acc: 0.0281 - val_loss: 15.6363 - val_acc: 0.0316
Epoch 4/20
392/392 [==============================] - 28s 71ms/step - loss: 15.4393 - acc: 0.0434 - val_loss: 15.6257 - val_acc: 0.0316
F1-Score = 0.032147937411095305
------ END MODEL ------


kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 100 | learn_rate=0.005 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 9s 24ms/step - loss: 17.3248 - acc: 0.0357 - val_loss: 17.2817 - val_acc: 0.0345
Epoch 2/20
392/392 [==============================] - 9s 22ms/step - loss: 17.0280 - acc: 0.0459 - val_loss: 17.1431 - val_acc: 0.0345
Epoch 3/20
392/392 [==